In [23]:
import pandas as pd
import numpy as np
import json
import os
from IPython.display import display

import scipy, scipy.io

import matplotlib
from matplotlib import *
from matplotlib import pyplot as plt
import itertools
from mpl_toolkits.axes_grid1 import make_axes_locatable
# pretty charting
import seaborn as sns
sns.set_palette('muted')
sns.set_style('darkgrid')
from natsort import natsorted, ns

%matplotlib inline

from operator import itemgetter

## sklearn imports
import sklearn.linear_model
import sklearn.cross_validation
import sklearn.tree
import sklearn.ensemble
import sklearn.preprocessing
import sklearn.feature_selection
import sklearn.pipeline
import sklearn.grid_search

In [2]:
def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    data = scipy.io.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

def _check_keys(dict):
    '''
    checks if entries in dictionary are mat-objects. If yes
    todict is called to change them to nested dictionaries
    '''
    for key in dict:
        if isinstance(dict[key], scipy.io.matlab.mio5_params.mat_struct):
            dict[key] = _todict(dict[key])
    return dict        

def _todict(matobj):
    '''
    A recursive function which constructs from matobjects nested dictionaries
    '''
    dict = {}
    for strg in matobj._fieldnames:
        elem = matobj.__dict__[strg]
        if isinstance(elem, scipy.io.matlab.mio5_params.mat_struct):
            dict[strg] = _todict(elem)
        elif isinstance(elem,np.ndarray):
            dict[strg] = _tolist(elem)
        else:
            dict[strg] = elem
    return dict

def _tolist(ndarray):
    '''
    A recursive function which constructs lists from cellarrays 
    (which are loaded as numpy ndarrays), recursing into the elements
    if they contain matobjects.
    '''
    elem_list = []            
    for sub_elem in ndarray:
        if isinstance(sub_elem, scipy.io.matlab.mio5_params.mat_struct):
            elem_list.append(_todict(sub_elem))
        elif isinstance(sub_elem,np.ndarray):
            elem_list.append(_tolist(sub_elem))
        else:
            elem_list.append(sub_elem)
    return elem_list

In [3]:
dataFile = '../Monkey X/clusters/pearsonRSpectralClustered_allconditions'
data = loadmat(dataFile)

In [9]:
indiceFile = '../Monkey X/clusters/cindices'
indices = loadmat(indiceFile)
indices = indices['clusters']
print "The indices for each neuron: ", indices

The indices for each neuron:  [3 3 3 1 3 3 3 3 3 1 3 3 3 3 3 1 4 3 3 2 3 3 2 3 3 3 3 3 3 1 3 1 3 3 4 4 3
 3 3 3 3 1 3 3 2 3 3 3 1 3 3 2 2 3 3 3 3 2 3 1 3 3 3 3 3 3 3 3 1 3 2 3 3 1
 3 3 3 1 3 3 1 3 1 3 3 3 3 3 3 3 4 3 3 3 3 3 3 3 3 3 4 3 3 3 3 1 2 3 3 3 3
 3 3 1 3 3 2 3 2 3 2 3 3 3 3 2 3 1 2 3 2 3 3 3 3 3 1 3 4 3 3 3 3 3 3 3 3 3
 1 3 3 3 3 2 3 3 4 3 3 3 3 3 3 3 3 3 3]


In [13]:
# get the neuron names in order
binNeuronDir = '../Monkey X/binned neurons 1ms/'
fileNames = []
for file in os.listdir(binNeuronDir):
    fileNames.append(file[0:len(file)-4])
print fileNames


['sig001a', 'sig001b', 'sig001c', 'sig003a', 'sig003b', 'sig003c', 'sig004a', 'sig004b', 'sig004c', 'sig004d', 'sig005a', 'sig005b', 'sig005c', 'sig005d', 'sig006a', 'sig006b', 'sig006c', 'sig007a', 'sig007b', 'sig007c', 'sig007d', 'sig008a', 'sig008b', 'sig008c', 'sig008d', 'sig009a', 'sig009b', 'sig010a', 'sig010b', 'sig011a', 'sig011b', 'sig012a', 'sig012b', 'sig014a', 'sig014b', 'sig014c', 'sig014d', 'sig015a', 'sig015b', 'sig020a', 'sig021a', 'sig021b', 'sig022a', 'sig022b', 'sig025a', 'sig027a', 'sig028a', 'sig033a', 'sig033b', 'sig033c', 'sig033d', 'sig034a', 'sig034b', 'sig034c', 'sig034d', 'sig035a', 'sig035b', 'sig036a', 'sig036b', 'sig036c', 'sig036d', 'sig037a', 'sig037b', 'sig037c', 'sig038a', 'sig038b', 'sig038c', 'sig038d', 'sig039a', 'sig039b', 'sig039c', 'sig040a', 'sig040b', 'sig041a', 'sig041b', 'sig041c', 'sig041d', 'sig042a', 'sig042b', 'sig043a', 'sig043b', 'sig046a', 'sig046b', 'sig046c', 'sig047a', 'sig047b', 'sig047c', 'sig047d', 'sig051a', 'sig052a', 'sig052b'

# analyze cluster 1

In [28]:
cluster_indices = [i for i,x in enumerate(indices) if x == 1]
print cluster_indices

cluster_neurons = []
for index in cluster_indices: # loop through each index
    cluster_neurons.append(fileNames[index])
    
print cluster_neurons

[3, 9, 15, 29, 31, 41, 48, 59, 68, 73, 77, 80, 82, 105, 113, 127, 136, 148]
['sig003a', 'sig004d', 'sig006b', 'sig011a', 'sig012a', 'sig021b', 'sig033b', 'sig036c', 'sig039a', 'sig041a', 'sig042a', 'sig043b', 'sig046b', 'sig075a', 'sig089b', 'sig097b', 'sig105a', 'sig115a']


In [36]:
for neuron in cluster_neurons:
    print neuron
    data = loadmat(os.path.join(binNeuronDir, neuron))
    binned_neuron = data['binned_neuron']
    push = binned_neuron['push']
    trials = push.keys()
    
    # just look at 1 trial
    extrial = push['trial1']
    
    print extrial.keys()
    print push.keys()
    
    break

sig003a
['trial69', 'trial68', 'trial61', 'trial60', 'trial63', 'trial62', 'trial65', 'trial64', 'trial67', 'trial66', 'trial89', 'trial88', 'trial83', 'trial82', 'trial81', 'trial80', 'trial87', 'trial86', 'trial85', 'trial84', 'trial76', 'trial77', 'trial74', 'trial75', 'trial72', 'trial73', 'trial70', 'trial71', 'trial78', 'trial79', 'trial98', 'trial99', 'trial90', 'trial91', 'trial92', 'trial93', 'trial94', 'trial95', 'trial96', 'trial97', 'trial112', 'trial113', 'trial111', 'trial116', 'trial117', 'trial114', 'trial115', 'trial11', 'trial12', 'trial13', 'trial14', 'trial15', 'trial16', 'trial17', 'trial18', 'trial19', 'trial109', 'trial108', 'trial105', 'trial104', 'trial107', 'trial106', 'trial101', 'trial100', 'trial103', 'trial102', 'trial29', 'trial28', 'trial25', 'trial24', 'trial27', 'trial26', 'trial21', 'trial20', 'trial23', 'trial22', 'trial49', 'trial48', 'trial47', 'trial46', 'trial45', 'trial43', 'trial42', 'trial41', 'trial40', 'trial38', 'trial39', 'trial32', 'trial

In [35]:
print binned_neuron.keys()

['push', 'pull', 'sphere', 'mallet']
